In [1]:
# Célula 1: Instalação e Imports (sem alterações)
!pip install -q google-generativeai
import google.generativeai as genai
from google.colab import userdata
# import time # Não precisamos mais dele para pausas forçadas aqui, mas pode ser útil depois

# --- COMENTÁRIO IMPORTANTE ---
# Lembre-se de adicionar sua GEMINI_API_KEY aos segredos do Colab
# (Ícone de chave na barra lateral esquerda)
# Nome do segredo: GEMINI_API_KEY
# Valor: Sua_API_Key_Aqui
# ---------------------------------

In [2]:
# Célula 1: Instalação e Imports (sem alterações)
# ... (código da Célula 1)

# Célula 2: Configuração da API Key e Funções do Agente (Atualizada)
# Use o modelo que funcionou!
MODELO_ESCOLHIDO = 'models/gemini-1.5-flash-latest' # <<-- USE ESTE MODELO
API_KEY_CONFIGURADA = False
GEMINI_MODEL = None

def configurar_api():
    global API_KEY_CONFIGURADA, GEMINI_MODEL
    try:
        api_key_value = userdata.get('GEMINI_API_KEY')
        if not api_key_value:
            print("API Key não encontrada nos segredos do Colab. Por favor, adicione-a.")
            print("Nome do segredo: GEMINI_API_KEY")
            return False

        genai.configure(api_key=api_key_value)
        GEMINI_MODEL = genai.GenerativeModel(MODELO_ESCOLHIDO)
        API_KEY_CONFIGURADA = True
        print(f"API Key configurada e modelo '{MODELO_ESCOLHIDO}' carregado com sucesso.")
        return True
    except Exception as e:
        print(f"Erro ao configurar a API Key ou carregar o modelo: {e}")
        if "429" in str(e):
            print("Isso pode ser um erro de quota. Tente novamente mais tarde.")
        API_KEY_CONFIGURADA = False
        return False

def obter_sugestao_produtividade(objetivo):
    if not API_KEY_CONFIGURADA or not GEMINI_MODEL:
        print("API não configurada.")
        return "Erro: API não configurada."

    # --- PROMPT REFINADO PARA SUGESTÕES (TAREFA 3) ---
    prompt = f"""Você é um coach de produtividade pessoal, otimista e focado em ação.
Sua tarefa é ajudar o usuário a começar seu objetivo principal para hoje, tornando-o menos intimidante.

Objetivo do usuário: '{objetivo}'.

Sugira 3 (TRÊS) passos INICIAIS, pequenos e acionáveis que o usuário possa realizar AGORA (nos próximos minutos) para dar o primeiro passo em direção a este objetivo.
Apresente-os como uma lista numerada (1., 2., 3.).
Cada passo deve ser muito concreto e levar no máximo 30 minutos para ser completado.
Seja encorajador e focado em iniciar.
"""
    try:
        print(f"\n[Agente] -> Pensando em como começar...") # Feedback para o usuário
        response = GEMINI_MODEL.generate_content(prompt)
        print("[Agente] -> Sugestões prontas.") # Feedback
        # Basic safety check and handling of potential empty or problematic responses
        if response and response.text:
             return response.text
        else:
             return "Não consegui gerar sugestões no momento. Tente reformular seu objetivo."

    except Exception as e:
        print(f"Erro ao gerar conteúdo: {e}")
        if "429" in str(e):
            return "[Agente] -> Desculpe, atingi um limite de solicitações. Por favor, tente um pouco mais tarde."
        return "[Agente] -> Desculpe, não consegui processar sua solicitação no momento."

def obter_dica_para_dificuldade(objetivo, dificuldade_descrita):
    if not API_KEY_CONFIGURADA or not GEMINI_MODEL:
        print("API não configurada.")
        return "Erro: API não configurada."

    # --- PROMPT REFINADO PARA DICAS (TAREFA 3) ---
    prompt = f"""Você é um mentor de produtividade empático e experiente, focado em remover bloqueios mentais ou práticos.
O usuário está trabalhando no objetivo: '{objetivo}'.
Ele descreveu a seguinte dificuldade/bloqueio: '{dificuldade_descrita}'.

Com base nesta dificuldade ESPECÍFICA, forneça UMA (UMA) dica prática, uma estratégia simples, uma perspectiva diferente ou uma palavra de encorajamento FORTE para ajudá-lo a superar esse obstáculo imediatamente.
Seja direto, compreensivo e focado em uma ação ou mudança de mindset para superar o bloqueio descrito.
A resposta deve ser um parágrafo curto, direto ao ponto.
"""
    try:
        print(f"[Agente] -> Entendendo sua dificuldade...") # Feedback
        response = GEMINI_MODEL.generate_content(prompt)
        print("[Agente] -> Dica pronta.") # Feedback
        if response and response.text:
            return response.text
        else:
             return "Não consegui gerar uma dica específica para essa dificuldade agora. Tente descrevê-la de outra forma."

    except Exception as e:
        print(f"Erro ao gerar dica para dificuldade: {e}")
        if "429" in str(e):
            return "[Agente] -> Desculpe, atingi um limite de solicitações. Por favor, tente um pouco mais tarde."
        return "[Agente] -> Desculpe, não consegui processar sua solicitação para a dificuldade no momento."

In [4]:
# Célula 3: Loop Principal de Interação do Agente (Atualizada para Tarefa 3)

if configurar_api():
    print("\n--- Bem-vindo ao seu Agente de Produtividade Pessoal! ---")
    print("[Agente] -> Olá! Estou aqui para te ajudar a focar no que é importante hoje.")

    # Loop simples para permitir mais de uma interação (opcional, mas adiciona fluidez)
    # Para simplificar no tempo, vamos manter o fluxo linear por enquanto.
    # Um loop real exigiria salvar o estado da conversa, o que é mais complexo para 4h.
    # Manteremos o fluxo: Objetivo -> Sugestões -> Dificuldade -> Dica -> Fim.

    # 1. Obter o objetivo do dia
    objetivo_do_dia = input("\n[Você] -> Qual é o seu principal objetivo ou tarefa mais importante para hoje? ")

    if objetivo_do_dia:
        print(f"[Agente] -> Certo. Seu foco para hoje é: {objetivo_do_dia}")

        sugestoes_iniciais = obter_sugestao_produtividade(objetivo_do_dia)
        print("\n[Agente] -> Aqui estão algumas sugestões para você começar:")
        # Imprimir linha por linha para melhor formatação
        for linha in sugestoes_iniciais.split('\n'):
            print(linha)

        # 2. Verificar dificuldades (Fluxo da Tarefa 2/3)
        print("\n" + "-" * 30) # Separador visual
        tem_dificuldade = input("[Agente] -> Você está encontrando alguma dificuldade específica ou bloqueio em relação a este objetivo ou aos primeiros passos? (s/n): ").lower()

        if tem_dificuldade == 's':
            descricao_dificuldade = input("[Você] -> Por favor, descreva brevemente sua dificuldade: ")
            if descricao_dificuldade:
                dica = obter_dica_para_dificuldade(objetivo_do_dia, descricao_dificuldade)
                print("\n[Agente] -> Entendo. Aqui está uma dica que pode te ajudar:")
                 # Imprimir linha por linha para melhor formatação
                for linha in dica.split('\n'):
                    print(linha)

                # Adicionar um pequeno follow-up (Tarefa 3)
                print("\n[Agente] -> Espero que esta dica seja útil! Lembre-se, dar o primeiro pequeno passo já cria momentum.")
                input("[Agente] -> Pronto para tentar? (Aperte Enter para finalizar a sessão) ") # Espera o usuário ler/processar

            else:
                print("[Agente] -> Ok. Se precisar de ajuda depois, me diga! Superar bloqueios faz parte do processo.")
        else:
            print("\n[Agente] -> Excelente! Fico feliz em saber que você está com clareza. Continue com o bom trabalho e foco!")
            input("[Agente] -> Pronto para agir? (Aperte Enter para finalizar a sessão) ") # Espera o usuário ler/processar


        print("\n[Agente] -> Lembre-se de fazer pausas e celebrar pequenas vitórias ao longo do dia!")

    else:
        print("[Agente] -> Nenhum objetivo fornecido. Que tal definir um para começar bem o dia na próxima vez?")

    print("\n--- Sessão com o Agente de Produtividade Finalizada ---")

else:
    print("\nNão foi possível iniciar o agente devido a problemas na configuração da API.")
    print("Verifique as mensagens de erro acima e as instruções para configurar a GEMINI_API_KEY.")

API Key configurada e modelo 'models/gemini-1.5-flash-latest' carregado com sucesso.

--- Bem-vindo ao seu Agente de Produtividade Pessoal! ---
[Agente] -> Olá! Estou aqui para te ajudar a focar no que é importante hoje.

[Você] -> Qual é o seu principal objetivo ou tarefa mais importante para hoje? Estudar para certificação em Investimentos CFP
[Agente] -> Certo. Seu foco para hoje é: Estudar para certificação em Investimentos CFP

[Agente] -> Pensando em como começar...
[Agente] -> Sugestões prontas.

[Agente] -> Aqui estão algumas sugestões para você começar:
E aí, futuro CFP!  "Estudar para a certificação CFP" pode parecer uma montanha, mas vamos desmontá-la em pedacinhos!  A chave é começar, e começar pequeno!  Vamos colocar você em movimento AGORA mesmo, com três passos MUITO simples que você consegue fazer nos próximos minutos:

1. **Identifique seu "primeiro pedacinho":** Abra seus materiais de estudo (livro, apostilas online, qualquer coisa!).  Escolha APENAS UM tópico, o mais